In [92]:
import numpy as np
from numpy.linalg import svd, norm, inv, pinv, qr, matrix_rank
from scipy.linalg import expm, orth
from scipy.spatial.distance import pdist
import scipy.optimize as opt
import matplotlib.pyplot as plt
import itertools
from itertools import combinations

In [93]:
def generate_near_symmetric_matrix(n, k, delta, spread_factor, seed=None):
    """
    Generate a near-symmetric matrix F with controlled spread factor
    
    FIXES:
    1. Ensure F has exactly rank k (not full rank)
    2. Control the spread factor properly
    3. Make the perturbation scale correctly with delta
    """
    if seed is not None:
        np.random.seed(seed)

    # 1) Generate random orthonormal bases U and V_base
    U_full, _ = np.linalg.qr(np.random.randn(n, n))
    U = U_full[:, :k]  # n × k
    
    # 2) Generate δ-skew perturbation of I_k (this part is correct)
    perturb = delta * np.random.randn(k, k)
    perturb = (perturb - perturb.T) / 2  # Make skew-symmetric
    Q = expm(perturb)  # Q ∈ O(k), ||Q-I||_2 ≈ δ

    # 3) Create V close to U via Q, then orthonormalize
    V_candidate = U @ Q  # This should give V close to U
    V, _ = np.linalg.qr(V_candidate)  # Orthonormalize to get exactly orthonormal V

    # 4) Check near-symmetry condition
    actual_delta = np.linalg.norm(U.T @ V - Q, 2)
    if actual_delta > 2 * delta:  # Allow some tolerance for QR effects
        print(f"Warning: δ-tolerance slightly violated: {actual_delta:.2e} > {delta:.2e}")

    # 5) Create singular values with controlled spread factor
    # spread_factor controls σ_1/σ_k ratio
    if spread_factor == 0:
        singular_values = np.ones(k)  # All equal singular values
    else:
        # Create logarithmic spread: σ_i = σ_max * (1/spread_factor)^((i-1)/(k-1))
        sigma_max = 1.0
        sigma_min = sigma_max / spread_factor
        if k == 1:
            singular_values = np.array([sigma_max])
        else:
            # Logarithmic spacing from sigma_max to sigma_min
            log_vals = np.linspace(np.log(sigma_max), np.log(sigma_min), k)
            singular_values = np.exp(log_vals)

    # 6) Assemble F = U @ S @ V^T (this ensures rank exactly k)
    S = np.diag(singular_values)
    F = U @ S @ V.T  # n × n matrix with rank exactly k

    # 7) Generate independent test matrix X 
    s = k + 1
    X_full, _ = np.linalg.qr(np.random.randn(n, s))
    X = X_full  # n × s

    # Verify F has correct rank
    computed_rank = np.linalg.matrix_rank(F, tol=1e-12)
    if computed_rank != k:
        raise RuntimeError(f"F has rank {computed_rank}, expected {k}")

    return F, X, U, V, actual_delta, Q

In [ ]:
def construct_omega_with_reorthogonalization(F, X, epsilon, num_samples=100, verbose=False):
    """
    Construct Omega set using SVD truncation followed by re-orthogonalization
    
    Strategy:
    1. Generate A_temp = F + E where E satisfies X^T E = 0
    2. Truncate A_temp to target rank via SVD
    3. Re-orthogonalize the result to restore X^T(A-F) = 0
    """
    n = F.shape[0]
    target_rank = np.linalg.matrix_rank(F, tol=1e-12)
    
    if verbose:
        print(f"SVD + Re-orthogonalization approach:")
        print(f"  F shape: {F.shape}, rank: {target_rank}")
        print(f"  X shape: {X.shape}")
        print(f"  epsilon: {epsilon:.2e}")
    
    # Compute null space projector for X^T
    P_X = X @ X.T  # Projects onto range(X)
    P_null = np.eye(n) - P_X  # Projects onto null(X^T)
    
    omega_matrices = []
    attempts = 0
    max_attempts = num_samples * 50
    
    while len(omega_matrices) < num_samples and attempts < max_attempts:
        attempts += 1
        
        # Step 1: Generate E in null space of X^T
        E_raw = np.random.randn(n, n)
        E_raw = (E_raw + E_raw.T) / 2  # Make symmetric
        E_projected = P_null @ E_raw @ P_null  # Project to null space
        
        # Scale E to have desired norm
        E_norm = np.linalg.norm(E_projected, 2)
        if E_norm < 1e-12:
            continue
            
        scale = epsilon * np.random.uniform(0.1, 0.8) / E_norm
        E = scale * E_projected
        
        # Step 2: Create A_temp and truncate via SVD
        A_temp = F + E
        U_temp, s_temp, Vt_temp = svd(A_temp, full_matrices=False)
        A_truncated = U_temp[:, :target_rank] @ np.diag(s_temp[:target_rank]) @ Vt_temp[:target_rank, :]
        
        # Step 3: Re-orthogonalize to restore X^T(A-F) = 0
        # We want to find A such that:
        # - A is close to A_truncated
        # - X^T(A - F) = 0
        # - rank(A) = target_rank
        
        # Method 3a: Project the difference back to null space
        diff = A_truncated - F
        diff_orth_violation = X.T @ diff
        
        # Remove the orthogonality violation
        # If X^T(A-F) = V, we want to subtract X @ V from (A-F)
        try:
            # Solve X^T @ correction = diff_orth_violation for correction
            correction = X @ diff_orth_violation  # This removes orthogonality violation
            A_corrected = A_truncated - correction
            
            # Step 4: Final rank correction (optional)
            # The correction might have changed the rank, so re-truncate if needed
            U_final, s_final, Vt_final = svd(A_corrected, full_matrices=False)
            A_final = U_final[:, :target_rank] @ np.diag(s_final[:target_rank]) @ Vt_final[:target_rank, :]
            
            # Check constraints
            distance = np.linalg.norm(A_final - F, 2)
            orthogonality_error = np.linalg.norm(X.T @ (A_final - F), 'fro')
            rank_A = np.linalg.matrix_rank(A_final, tol=1e-12)
            
            distance_ok = distance <= epsilon * 1.05  # Small tolerance
            orthogonality_ok = orthogonality_error <= 1e-8
            rank_ok = rank_A == target_rank
            
            if distance_ok and orthogonality_ok and rank_ok:
                omega_matrices.append(A_final)
                if verbose and len(omega_matrices) <= 3:
                    print(f"  Success {len(omega_matrices)}: "
                          f"||A-F||={distance:.2e}, "
                          f"||X^T(A-F)||={orthogonality_error:.2e}, "
                          f"rank={rank_A}")
            
        except np.linalg.LinAlgError:
            continue
    
    if verbose:
        success_rate = len(omega_matrices) / attempts * 100
        print(f"Generated {len(omega_matrices)} matrices in {attempts} attempts ({success_rate:.1f}% success)")
    
    return omega_matrices

def construct_omega_iterative_projection(F, X, epsilon, num_samples=100, verbose=False):
    """
    Alternative: Iterative projection between rank and orthogonality constraints
    """
    n = F.shape[0]
    target_rank = np.linalg.matrix_rank(F, tol=1e-12)
    
    if verbose:
        print(f"Iterative projection approach:")
        print(f"  Target rank: {target_rank}, epsilon: {epsilon:.2e}")
    
    # Projectors
    P_null = np.eye(n) - X @ X.T
    
    omega_matrices = []
    
    for sample in range(num_samples):
        # Start with random perturbation
        E_init = np.random.randn(n, n)
        E_init = (E_init + E_init.T) / 2
        E_norm = np.linalg.norm(E_init, 2)
        if E_norm < 1e-12:
            continue
            
        scale = epsilon * np.random.uniform(0.1, 0.5) / E_norm
        A = F + scale * E_init
        
        # Iterative projection (alternating projections)
        for iteration in range(10):  # Maximum iterations
            # Project to satisfy orthogonality constraint
            diff = A - F
            orth_violation = X.T @ diff
            correction = X @ orth_violation
            A = A - correction
            
            # Project to satisfy rank constraint
            U, s, Vt = svd(A, full_matrices=False)
            A = U[:, :target_rank] @ np.diag(s[:target_rank]) @ Vt[:target_rank, :]
            
            # Check convergence
            distance = np.linalg.norm(A - F, 2)
            orthogonality_error = np.linalg.norm(X.T @ (A - F), 'fro')
            
            if orthogonality_error < 1e-8 and distance <= epsilon * 1.05:
                break
        
        # Final check
        distance = np.linalg.norm(A - F, 2)
        orthogonality_error = np.linalg.norm(X.T @ (A - F), 'fro')
        rank_A = np.linalg.matrix_rank(A, tol=1e-12)
        
        if distance <= epsilon * 1.05 and orthogonality_error <= 1e-8 and rank_A == target_rank:
            omega_matrices.append(A)
            if verbose and len(omega_matrices) <= 3:
                print(f"  Success {len(omega_matrices)}: "
                      f"||A-F||={distance:.2e}, "
                      f"||X^T(A-F)||={orthogonality_error:.2e}, "
                      f"rank={rank_A}")
    
    if verbose:
        print(f"Generated {len(omega_matrices)}/{num_samples} matrices")
    
    return omega_matrices

def construct_omega_manifold_projection(F, X, epsilon, num_samples=100, verbose=False):
    """
    Advanced approach: Project onto the intersection of manifolds
    """
    n = F.shape[0]
    target_rank = np.linalg.matrix_rank(F, tol=1e-12)
    
    if verbose:
        print(f"Manifold projection approach:")
        print(f"  Target rank: {target_rank}, epsilon: {epsilon:.2e}")
    
    # Get SVD of F for manifold structure
    U_F, S_F, Vt_F = svd(F, full_matrices=False)
    U_F = U_F[:, :target_rank]
    S_F = S_F[:target_rank]
    Vt_F = Vt_F[:target_rank, :]
    
    omega_matrices = []
    
    for sample in range(num_samples):
        # Parametrize rank-r matrices as A = U @ S @ V^T
        # where U and V are close to U_F and V_F^T
        
        # Small perturbations to U_F and V_F^T
        delta_U = np.random.randn(n, target_rank) * 0.01
        delta_V = np.random.randn(n, target_rank) * 0.01
        delta_S = np.random.randn(target_rank) * epsilon * 0.1
        
        U_new = U_F + delta_U
        U_new, _ = qr(U_new)  # Re-orthogonalize
        
        V_new = Vt_F.T + delta_V
        V_new, _ = qr(V_new)  # Re-orthogonalize
        
        S_new = S_F + delta_S
        S_new = np.maximum(S_new, 0.01)  # Keep positive
        
        # Construct candidate matrix
        A_candidate = U_new @ np.diag(S_new) @ V_new.T
        
        # Project to satisfy orthogonality constraint
        diff = A_candidate - F
        orth_violation = X.T @ diff
        correction = X @ orth_violation
        A = A_candidate - correction
        
        # Check if we still have the right rank (correction might have changed it)
        rank_A = np.linalg.matrix_rank(A, tol=1e-12)
        if rank_A != target_rank:
            # Re-project to rank manifold
            U_corr, s_corr, Vt_corr = svd(A, full_matrices=False)
            A = U_corr[:, :target_rank] @ np.diag(s_corr[:target_rank]) @ Vt_corr[:target_rank, :]
        
        # Final constraint check
        distance = np.linalg.norm(A - F, 2)
        orthogonality_error = np.linalg.norm(X.T @ (A - F), 'fro')
        rank_A = np.linalg.matrix_rank(A, tol=1e-12)
        
        if distance <= epsilon * 1.05 and orthogonality_error <= 1e-6 and rank_A == target_rank:
            omega_matrices.append(A)
            if verbose and len(omega_matrices) <= 3:
                print(f"  Success {len(omega_matrices)}: "
                      f"||A-F||={distance:.2e}, "
                      f"||X^T(A-F)||={orthogonality_error:.2e}, "
                      f"rank={rank_A}")
    
    if verbose:
        print(f"Generated {len(omega_matrices)}/{num_samples} matrices")
    
    return omega_matrices

def test_reorthogonalization_approaches(F, X, epsilon):
    """
    Test all re-orthogonalization approaches
    """
    print(f"\n=== TESTING RE-ORTHOGONALIZATION APPROACHES ===")
    print(f"F rank: {np.linalg.matrix_rank(F)}, epsilon: {epsilon:.2e}")
    
    approaches = [
        ("Strict Nullspace Construction", construct_omega_set_strict)
    ]
    
    best_result = []
    best_count = 0
    
    for name, func in approaches:
        print(f"\nTesting {name}:")
        try:
            result = func(F, X, epsilon, num_samples=20, verbose=True)
            if len(result) > best_count:
                best_result = result
                best_count = len(result)
                print(f"  -> New best: {len(result)} matrices")
        except Exception as e:
            print(f"  -> Failed with error: {e}")
    
    print(f"\nBest approach generated {best_count} matrices")
    return best_result

def construct_omega_set_strict(F, X, epsilon, num_samples=100, verbose=False):
    """
    Construct Omega_{F,X}^epsilon:
      - X^T(A - F) = 0
      - rank(A) = rank(F)
      - ||A - F||_2 <= epsilon
    """

    n = F.shape[0]
    target_rank = matrix_rank(F, tol=1e-12)
    omega_matrices = []

    # Projector to nullspace of X^T
    # X: (n, s)
    Ux, _, _ = svd(X, full_matrices=True)
    s = X.shape[1]
    P_null = np.eye(n) - Ux[:, :s] @ Ux[:, :s].T

    # Always add F itself (trivial Omega member)
    omega_matrices.append(F.copy())
    if verbose:
        print(f"  Added F itself (trivially valid)")

    attempts = 0
    max_attempts = num_samples * 10

    while len(omega_matrices) < num_samples and attempts < max_attempts:
        attempts += 1

        # Random symmetric perturbation in null(X^T)
        E = np.random.randn(n, n)
        E = (E + E.T) / 2
        E_proj = P_null @ E @ P_null  # Projects into null(X^T) both ways

        E_proj = E_proj / (np.linalg.norm(E_proj, 2) + 1e-12)  # Normalize

        # Scale the perturbation to be within epsilon (fraction of max)
        scale = np.random.uniform(0, 1) * epsilon
        N = scale * E_proj

        A = F + N

        # Check constraints
        distance = np.linalg.norm(A - F, 2)
        orthogonality_error = np.linalg.norm(X.T @ (A - F), 'fro')
        rank_A = matrix_rank(A, tol=1e-10)

        # Small perturbations from null(X^T) will almost always preserve rank
        # But check anyway:
        distance_ok = distance <= epsilon * 1.01
        orthogonality_ok = orthogonality_error <= 1e-8
        rank_ok = rank_A == target_rank

        if distance_ok and orthogonality_ok and rank_ok:
            omega_matrices.append(A)
            if verbose and len(omega_matrices) <= 3:
                print(f"  Success {len(omega_matrices)}: "
                      f"||A-F||={distance:.2e}, "
                      f"||X^T(A-F)||={orthogonality_error:.2e}, "
                      f"rank={rank_A}")

    if verbose:
        print(f"Generated {len(omega_matrices)}/{num_samples} valid Omega members.")
    return omega_matrices

def construct_omega_set_strict_relaxed(F, X, epsilon, num_samples=10000, verbose=False):
    """
    Like construct_omega_set_strict, but relaxes the rank check slightly (allows tiny singular values).
    """
    n = F.shape[0]
    target_rank = np.linalg.matrix_rank(F, tol=1e-12)
    omega_matrices = [F.copy()]

    Ux, _, _ = np.linalg.svd(X, full_matrices=True)
    s = X.shape[1]
    P_null = np.eye(n) - Ux[:, :s] @ Ux[:, :s].T

    attempts = 0
    max_attempts = num_samples * 20
    while len(omega_matrices) < num_samples and attempts < max_attempts:
        attempts += 1

        E = np.random.randn(n, n)
        E = (E + E.T) / 2
        E_proj = P_null @ E @ P_null
        E_proj = E_proj / (np.linalg.norm(E_proj, 2) + 1e-12)
        scale = np.random.uniform(0, 1) * epsilon
        N = scale * E_proj
        A = F + N

        # "Relaxed" rank check: allow near-zero singular values (numerical rank)
        svals = np.linalg.svd(A, compute_uv=False)
        numerical_rank = np.sum(svals > 1e-8)
        orthogonality_error = np.linalg.norm(X.T @ (A - F), 'fro')
        distance = np.linalg.norm(A - F, 2)

        if (
            distance <= epsilon * 1.01
            and orthogonality_error <= 1e-8
            and numerical_rank == target_rank
        ):
            omega_matrices.append(A)
            if verbose and len(omega_matrices) <= 3:
                print(f"  Success {len(omega_matrices)}: ||A-F||={distance:.2e}, rank={numerical_rank}, ||Xᵗ(A-F)||={orthogonality_error:.2e}")

    if verbose:
        print(f"Generated {len(omega_matrices)}/{num_samples} valid Omega members.")
    return omega_matrices

def empirical_supremum_over_omega(F, X, epsilon, num_samples=10000, verbose=False):
    """
    Generate many matrices in Omega_{F,X}^epsilon and compute empirical supremum of ||A-B||.
    """
    omega = construct_omega_set_strict_relaxed(F, X, epsilon, num_samples=num_samples, verbose=verbose)
    print(f"Generated {len(omega)} Omega matrices.")

    # Compute supremum ||A-B||_2 (spectral norm) over all pairs
    max_dist = 0
    argmax = (0, 0)
    for i in range(len(omega)):
        for j in range(i+1, len(omega)):
            dist = np.linalg.norm(omega[i] - omega[j], 2)
            if dist > max_dist:
                max_dist = dist
                argmax = (i, j)
    print(f"Empirical supremum ||A-B||_2 = {max_dist:.4e} over {num_samples} Omega samples.")
    print(f"Pair achieving supremum: {argmax}")
    return omega, max_dist, argmax

In [95]:
"""spread_factors = [1,2,3,4,5,6,7,8,9]
for sf in spread_factors:
    F, X, U, V, actual_delta, Q = generate_near_symmetric_matrix(
        n=50,
        k=10,
        delta=delta,
        spread_factor=sf,
        seed=None
    )
    omega_matrices = test_reorthogonalization_approaches(F, X, epsilon)"""

'spread_factors = [1,2,3,4,5,6,7,8,9]\nfor sf in spread_factors:\n    F, X, U, V, actual_delta, Q = generate_near_symmetric_matrix(\n        n=50,\n        k=10,\n        delta=delta,\n        spread_factor=sf,\n        seed=None\n    )\n    omega_matrices = test_reorthogonalization_approaches(F, X, epsilon)'

In [96]:
import numpy as np
import matplotlib.pyplot as plt

# BALANCED APPROACH: Find optimal X size that balances null space vs conditioning

def analyze_x_size_tradeoff(F, X_full, epsilon, delta, verbose=True):
    """
    Analyze the tradeoff between null space size and theorem condition
    as we vary the number of columns in X
    """
    n = F.shape[0]
    k = np.linalg.matrix_rank(F, tol=1e-12)
    max_cols = min(X_full.shape[1], n-1)  # Can't use all columns
    
    # Get SVD components of F
    U_F, S_F, Vt_F = np.linalg.svd(F, full_matrices=False)
    V_F = Vt_F[:k, :].T
    
    results = {
        'x_cols': [],
        'null_dims': [],
        'condition_numbers': [],
        'theorem_conditions': [],
        'feasible': []
    }
    
    if verbose:
        print(f"ANALYZING X SIZE TRADEOFF:")
        print(f"  F: {F.shape}, rank {k}")
        print(f"  Available X columns: {max_cols}")
        print(f"  epsilon: {epsilon:.3f}, delta: {delta:.1e}")
        print()
    
    # Test different numbers of X columns
    for num_cols in range(k, max_cols + 1):
        X = X_full[:, :num_cols]
        null_dim = n - np.linalg.matrix_rank(X, tol=1e-12)
        
        # Compute theorem condition
        XTV_F = X.T @ V_F
        rank_XTV_F = np.linalg.matrix_rank(XTV_F, tol=1e-12)
        
        if rank_XTV_F == k:  # Full rank condition
            sigma_vals = np.linalg.svd(XTV_F, compute_uv=False)
            c = sigma_vals[0] / (sigma_vals[-1]**2) if sigma_vals[-1] > 1e-12 else float('inf')
            theorem_cond = c * (np.sqrt(2*epsilon) + np.sqrt(2*delta))
            feasible = theorem_cond < 1
        else:
            c = float('inf')
            theorem_cond = float('inf')
            feasible = False
        
        results['x_cols'].append(num_cols)
        results['null_dims'].append(null_dim)
        results['condition_numbers'].append(c)
        results['theorem_conditions'].append(theorem_cond)
        results['feasible'].append(feasible)
        
        if verbose:
            status = "✓ FEASIBLE" if feasible else "✗ INFEASIBLE"
            print(f"  X cols: {num_cols:2d} | Null dim: {null_dim:2d} | c: {c:6.2f} | "
                  f"Theorem: {theorem_cond:6.3f} | {status}")
    
    return results

def find_best_x_configuration(F, X_full, epsilon, delta):
    """
    Find the best X configuration that satisfies theorem conditions
    """
    analysis = analyze_x_size_tradeoff(F, X_full, epsilon, delta, verbose=True)
    
    # Find configurations that satisfy theorem condition
    feasible_configs = []
    for i, feasible in enumerate(analysis['feasible']):
        if feasible:
            feasible_configs.append({
                'x_cols': analysis['x_cols'][i],
                'null_dim': analysis['null_dims'][i],
                'condition_number': analysis['condition_numbers'][i],
                'theorem_condition': analysis['theorem_conditions'][i]
            })
    
    if not feasible_configs:
        print("\n❌ NO FEASIBLE CONFIGURATIONS FOUND!")
        print("   The theorem conditions cannot be satisfied with current parameters.")
        print("   Try: larger epsilon, smaller delta, or different F/X generation.")
        return None, analysis
    
    # Choose configuration with largest null space among feasible ones
    best_config = max(feasible_configs, key=lambda x: x['null_dim'])
    
    print(f"\n✅ BEST FEASIBLE CONFIGURATION:")
    print(f"   X columns: {best_config['x_cols']}")
    print(f"   Null space dimension: {best_config['null_dim']}")
    print(f"   Condition number c: {best_config['condition_number']:.3f}")
    print(f"   Theorem condition: {best_config['theorem_condition']:.3f} < 1")
    
    return best_config, analysis

def test_omega_with_optimal_x(spread_factors, n, k, delta, epsilon):
    """
    Test omega construction with optimally chosen X size
    """
    print("="*80)
    print("OPTIMAL X SIZE OMEGA DEBUGGING")
    print("="*80)
    print(f"Parameters: n={n}, k={k}, delta={delta:.1e}, epsilon={epsilon:.1e}")
    print("Strategy: Find optimal X size for each case")
    print("="*80)
    
    successful_tests = 0
    all_results = []
    
    for i, sf in enumerate(spread_factors):
        print(f"\n{'='*20} SPREAD FACTOR {sf} ({'Test {}/{}'.format(i+1, len(spread_factors))}) {'='*20}")
        
        try:
            # Generate base matrices
            F, X_full, U, V, actual_delta, Q = generate_near_symmetric_matrix(
                n=n, k=k, delta=delta, spread_factor=sf, seed=42+i
            )
            
            print(f"Generated F (rank {np.linalg.matrix_rank(F)}) and X_full {X_full.shape}")
            
            # Find optimal X configuration
            best_config, analysis = find_best_x_configuration(F, X_full, epsilon, actual_delta)
            
            if best_config is None:
                print("❌ No feasible X configuration - skipping")
                all_results.append({'spread_factor': sf, 'success': False, 'matrices_generated': 0})
                continue
            
            # Use optimal X
            X = X_full[:, :best_config['x_cols']]
            print(f"\nUsing X with {X.shape[1]} columns (null space dim: {best_config['null_dim']})")
            
            # Test omega construction with optimal X
            print(f"\nTesting Omega Construction:")
            
            # Test only the iterative projection method (it was working before)
            #matrices = construct_omega_iterative_projection(F, X, epsilon, num_samples=20, verbose=True)
            matrices = construct_omega_set_strict_relaxed(F, X, epsilon, num_samples=20, verbose=True)
            
            if len(matrices) > 0:
                print(f"🎉 SUCCESS! Generated {len(matrices)} matrices")
                successful_tests += 1
                
                # Check first matrix constraints
                A = matrices[0]
                rank_A = np.linalg.matrix_rank(A, tol=1e-12)
                sketching_error = np.linalg.norm(A @ X - F @ X, 'fro')
                distance = np.linalg.norm(A - F, 2)
                
                print(f"   First matrix constraints:")
                print(f"     Rank: {rank_A} == {np.linalg.matrix_rank(F)} ✓" if rank_A == np.linalg.matrix_rank(F) else f"     Rank: {rank_A} != {np.linalg.matrix_rank(F)} ✗")
                print(f"     Sketching error: {sketching_error:.2e}")
                print(f"     Distance: {distance:.4f} ≤ {epsilon}")
                
                # Compute diameter if multiple matrices
                if len(matrices) >= 2:
                    max_diameter = 0
                    for i in range(len(matrices)):
                        for j in range(i+1, len(matrices)):
                            dist = np.linalg.norm(matrices[i] - matrices[j], 2)
                            max_diameter = max(max_diameter, dist)
                    print(f"     Omega set diameter: {max_diameter:.4e}")
                
                all_results.append({
                    'spread_factor': sf, 
                    'success': True, 
                    'matrices_generated': len(matrices),
                    'x_cols': best_config['x_cols'],
                    'null_dim': best_config['null_dim'],
                    'theorem_condition': best_config['theorem_condition']
                })
            else:
                print(f"❌ No matrices generated even with optimal X")
                all_results.append({'spread_factor': sf, 'success': False, 'matrices_generated': 0})
                
        except Exception as e:
            print(f"ERROR for spread_factor {sf}: {e}")
            import traceback
            traceback.print_exc()
            all_results.append({'spread_factor': sf, 'success': False, 'matrices_generated': 0})
    
    # Final summary
    print(f"\n{'='*80}")
    print("FINAL RESULTS")
    print("="*80)
    
    print(f"Successful tests: {successful_tests}/{len(spread_factors)}")
    
    if successful_tests > 0:
        print(f"\nSuccessful configurations:")
        for result in all_results:
            if result['success']:
                print(f"  Spread factor {result['spread_factor']}: "
                      f"{result['matrices_generated']} matrices, "
                      f"X cols: {result.get('x_cols', '?')}, "
                      f"null dim: {result.get('null_dim', '?')}")
        
        print(f"\n✅ SUCCESS! Found working Omega set construction.")
        print(f"   The key was finding the right balance between null space size")
        print(f"   and theorem condition satisfaction.")
    else:
        print(f"\n❌ NO SUCCESS across all configurations.")
        print(f"   Possible issues:")
        print(f"   1. epsilon = {epsilon} may still be too small")
        print(f"   2. The problem parameters (n={n}, k={k}) may be too challenging")
        print(f"   3. The constraints may be fundamentally incompatible")
        
        print(f"\n   RECOMMENDED NEXT STEPS:")
        print(f"   - Try epsilon = 0.5 or 1.0 (much larger)")
        print(f"   - Try smaller problem: n=10, k=2")
        print(f"   - Check if F itself satisfies constraints")
    
    return all_results

# MAIN EXECUTION
if __name__ == "__main__":
    # Test parameters
    spread_factors = [2, 3, 4, 5, 6]
    delta = 1e-3
    epsilon = 1  # Try this first
    n, k = 15, 3
    
    print("TESTING WITH CURRENT PARAMETERS FIRST:")
    results = test_omega_with_optimal_x(spread_factors, n, k, delta, epsilon)
    
    # If no success, try larger epsilon
    successful = sum(1 for r in results if r['success'])
    if successful == 0:
        print(f"\n\n" + "="*80)
        print("TRYING WITH LARGER EPSILON")
        print("="*80)
        epsilon_larger = 0.5
        print(f"Increasing epsilon from {epsilon} to {epsilon_larger}")
        results_2 = test_omega_with_optimal_x(spread_factors, n, k, delta, epsilon_larger)
        
        successful_2 = sum(1 for r in results_2 if r['success'])
        if successful_2 == 0:
            print(f"\n\nStill no success with epsilon = {epsilon_larger}")
            print(f"The Omega set construction appears to be fundamentally challenging")
            print(f"for these parameters. This suggests the theoretical bounds may be")
            print(f"very tight or the set may be extremely small.")

TESTING WITH CURRENT PARAMETERS FIRST:
OPTIMAL X SIZE OMEGA DEBUGGING
Parameters: n=15, k=3, delta=1.0e-03, epsilon=1.0e+00
Strategy: Find optimal X size for each case

==================== SPREAD FACTOR 2 (Test 1/5) ====================
Generated F (rank 3) and X_full (15, 4)
ANALYZING X SIZE TRADEOFF:
  F: (15, 15), rank 3
  Available X columns: 4
  epsilon: 1.000, delta: 3.6e-16

  X cols:  3 | Null dim: 12 | c:  19.10 | Theorem: 27.013 | ✗ INFEASIBLE
  X cols:  4 | Null dim: 11 | c:   3.78 | Theorem:  5.339 | ✗ INFEASIBLE

❌ NO FEASIBLE CONFIGURATIONS FOUND!
   The theorem conditions cannot be satisfied with current parameters.
   Try: larger epsilon, smaller delta, or different F/X generation.
❌ No feasible X configuration - skipping

==================== SPREAD FACTOR 3 (Test 2/5) ====================
Generated F (rank 3) and X_full (15, 4)
ANALYZING X SIZE TRADEOFF:
  F: (15, 15), rank 3
  Available X columns: 4
  epsilon: 1.000, delta: 3.6e-16

  X cols:  3 | Null dim: 12 | c: 

In [97]:
import numpy as np

def test_trivial_omega_membership():
    """
    Test if F trivially satisfies all Omega set constraints.
    This MUST work - if it doesn't, there's a fundamental bug.
    """
    print("="*80)
    print("TRIVIAL OMEGA SET MEMBERSHIP TEST")
    print("="*80)
    print("Testing: Does F ∈ Ω_{F,X}^ε for ANY reasonable ε?")
    print("This should ALWAYS be true by definition!")
    print("="*80)
    
    # Test multiple configurations
    test_configs = [
        (15, 3, 9, 0.1),   # Original problem
        (15, 3, 13, 0.1),  # More X columns
        (10, 2, 5, 0.1),   # Smaller problem
        (8, 2, 3, 0.1),    # Even smaller
    ]
    
    for n, k, x_cols, epsilon in test_configs:
        print(f"\n{'='*20} TEST: n={n}, k={k}, X_cols={x_cols}, ε={epsilon} {'='*20}")
        
        try:
            # Generate matrices
            F, X_full, U, V, actual_delta, Q = generate_near_symmetric_matrix(
                n=n, k=k, delta=1e-3, spread_factor=3, seed=42
            )
            
            X = X_full[:, :x_cols]
            
            print(f"Generated:")
            print(f"  F: {F.shape}, rank {np.linalg.matrix_rank(F, tol=1e-12)}")
            print(f"  X: {X.shape}, rank {np.linalg.matrix_rank(X, tol=1e-12)}")
            print(f"  Actual delta: {actual_delta:.2e}")
            print(f"  Null space dimension: {n - np.linalg.matrix_rank(X)}")
            
            print(f"\nTesting if F ∈ Ω_{{F,X}}^{epsilon}:")
            
            # CONSTRAINT 1: rank(A) = rank(F)
            rank_F = np.linalg.matrix_rank(F, tol=1e-12)
            rank_A = np.linalg.matrix_rank(F, tol=1e-12)  # A = F in this test
            constraint1_ok = (rank_A == rank_F)
            
            print(f"  1. Rank constraint: rank(F) = {rank_A} == {rank_F} = rank(F) {'✓' if constraint1_ok else '✗'}")
            
            # CONSTRAINT 2: AX = FX (sketching constraint)
            AX = F @ X  # A = F
            FX = F @ X
            sketching_error = np.linalg.norm(AX - FX, 'fro')
            constraint2_ok = sketching_error < 1e-12
            
            print(f"  2. Sketching: ||FX - FX||_F = {sketching_error:.2e} {'✓' if constraint2_ok else '✗'}")
            
            # CONSTRAINT 3: ||A - F||_2 ≤ ε
            distance = np.linalg.norm(F - F, 2)  # A = F
            constraint3_ok = distance <= epsilon
            
            print(f"  3. Distance: ||F - F||_2 = {distance:.2e} ≤ {epsilon} {'✓' if constraint3_ok else '✗'}")
            
            # CONSTRAINT 4: A is δ-near-symmetric
            # For F itself, this is true by construction
            constraint4_ok = True  # F is δ-near-symmetric by construction
            
            print(f"  4. Near-symmetry: F is δ-near-symmetric by construction {'✓' if constraint4_ok else '✗'}")
            
            # Overall check
            all_constraints_ok = constraint1_ok and constraint2_ok and constraint3_ok and constraint4_ok
            
            print(f"\n  RESULT: F ∈ Ω_{{F,X}}^{epsilon}: {'✅ YES' if all_constraints_ok else '❌ NO'}")
            
            if all_constraints_ok:
                print(f"  ✅ SUCCESS: F trivially satisfies all Omega constraints!")
                
                # Now test if our construction algorithm can find F
                print(f"\n  Testing if construction algorithms can find F...")
                
                # Test each construction method
                methods_to_test = [
    ("Strict Nullspace Construction", lambda: construct_omega_set_strict_relaxed(F, X, epsilon, num_samples=20, verbose=False))
]
                
                found_any = False
                for method_name, method_func in methods_to_test:
                    try:
                        matrices = method_func()
                        count = len(matrices)
                        print(f"    {method_name}: Generated {count} matrices")
                        
                        if count > 0:
                            found_any = True
                            # Check if any generated matrix is close to F
                            distances_to_F = [np.linalg.norm(A - F, 'fro') for A in matrices]
                            min_dist = min(distances_to_F)
                            print(f"      Closest matrix to F: distance = {min_dist:.4e}")
                            
                            if min_dist < 0.01:  # Very close to F
                                print(f"      🎉 Found matrix very close to F!")
                        
                    except Exception as e:
                        print(f"    {method_name}: ERROR - {e}")
                
                if not found_any:
                    print(f"    ❌ CRITICAL BUG: No construction method found ANY matrices")
                    print(f"       even though F ∈ Ω trivially!")
                    print(f"       This indicates a fundamental algorithm problem.")
                else:
                    print(f"    ✅ At least one method generates matrices")
                    
            else:
                print(f"  🚨 CRITICAL BUG: F does not satisfy its own Omega constraints!")
                print(f"     This should be impossible - there's a fundamental error.")
                
                if not constraint1_ok:
                    print(f"     - Rank issue: Implementation bug in rank calculation")
                if not constraint2_ok:
                    print(f"     - Sketching issue: FX != FX (numerical error?)")
                if not constraint3_ok:
                    print(f"     - Distance issue: ||F-F|| > 0 (should be exactly 0)")
                if not constraint4_ok:
                    print(f"     - Near-symmetry issue: F not δ-near-symmetric")
                
        except Exception as e:
            print(f"ERROR in configuration n={n}, k={k}: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"\n{'='*80}")
    print("TRIVIAL TEST SUMMARY")
    print("="*80)
    print("If F ∈ Ω trivially but algorithms can't find matrices:")
    print("  → Algorithm implementation bugs")
    print("If F ∉ Ω trivially:")
    print("  → Fundamental constraint definition errors")
    print("If both work:")
    print("  → Problem is with parameter regimes, not algorithms")

def debug_construction_algorithm_on_F():
    """
    Debug why construction algorithms might fail even when F ∈ Ω trivially
    """
    print(f"\n{'='*80}")
    print("DEBUGGING CONSTRUCTION ALGORITHMS WITH F")
    print("="*80)
    
    # Use a configuration we know works
    F, X_full, U, V, actual_delta, Q = generate_near_symmetric_matrix(
        n=10, k=2, delta=1e-3, spread_factor=2, seed=42
    )
    X = X_full[:, :3]  # Small null space but should still work
    epsilon = 1.0  # Large epsilon
    
    print(f"Configuration: F {F.shape} rank {np.linalg.matrix_rank(F)}, X {X.shape}, ε={epsilon}")
    
    # Manually trace through iterative projection algorithm
    print(f"\n=== MANUAL ITERATIVE PROJECTION TRACE ===")
    
    rank_F = np.linalg.matrix_rank(F, tol=1e-12)
    print(f"Target rank: {rank_F}")
    
    # Step 1: Generate random perturbation in null space
    U_X, S_X, Vt_X = np.linalg.svd(X, full_matrices=True)
    null_space_basis = U_X[:, X.shape[1]:]  # Null space of X
    print(f"Null space basis shape: {null_space_basis.shape}")
    
    # Try a few random perturbations
    for attempt in range(5):
        print(f"\n  Attempt {attempt + 1}:")
        
        # Random perturbation in null space
        alpha = np.random.randn(null_space_basis.shape[1]) * 0.01  # Small perturbation
        perturbation = null_space_basis @ alpha
        A_perturbed = F + perturbation.reshape(F.shape[0], 1) @ np.ones((1, F.shape[1]))
        
        # Check constraints before any projection
        rank_A = np.linalg.matrix_rank(A_perturbed, tol=1e-12)
        sketching_error = np.linalg.norm(A_perturbed @ X - F @ X, 'fro')
        distance = np.linalg.norm(A_perturbed - F, 2)
        
        print(f"    Before projection:")
        print(f"      Rank: {rank_A} (target: {rank_F})")
        print(f"      Sketching error: {sketching_error:.2e}")
        print(f"      Distance: {distance:.4f} (limit: {epsilon})")
        
        # If distance already too large, this explains failure
        if distance > epsilon:
            print(f"      ❌ Distance constraint violated before projection!")
            print(f"         This suggests epsilon is too small for meaningful perturbations")
        elif sketching_error > 1e-6:
            print(f"      ❌ Sketching constraint violated before projection!")
            print(f"         This suggests perturbation method is wrong")
        else:
            print(f"      ✅ Constraints satisfied before projection")
            
            # Now try to project to correct rank
            try:
                U_A, S_A, Vt_A = np.linalg.svd(A_perturbed, full_matrices=False)
                
                # Truncate to rank k
                A_projected = U_A[:, :rank_F] @ np.diag(S_A[:rank_F]) @ Vt_A[:rank_F, :]
                
                # Check final constraints
                rank_final = np.linalg.matrix_rank(A_projected, tol=1e-12)
                sketching_final = np.linalg.norm(A_projected @ X - F @ X, 'fro')
                distance_final = np.linalg.norm(A_projected - F, 2)
                
                print(f"      After rank projection:")
                print(f"        Rank: {rank_final} (target: {rank_F})")
                print(f"        Sketching error: {sketching_final:.2e}")
                print(f"        Distance: {distance_final:.4f} (limit: {epsilon})")
                
                all_ok = (rank_final == rank_F and 
                         sketching_final < 1e-6 and 
                         distance_final <= epsilon)
                
                if all_ok:
                    print(f"        🎉 SUCCESS: Found valid matrix!")
                    return True
                else:
                    print(f"        ❌ Some constraint failed after projection")
                    
            except Exception as e:
                print(f"      ERROR in projection: {e}")
    
    print(f"\n❌ Manual trace also failed - suggests fundamental algorithm issue")
    return False

# Run the tests
if __name__ == "__main__":
    print("Testing if F trivially belongs to its own Omega set...")
    test_trivial_omega_membership()
    
    print("\nDebugging construction algorithms...")
    success = debug_construction_algorithm_on_F()
    
    if not success:
        print(f"\n🚨 CONCLUSION: Even manual algorithm fails")
        print(f"   This suggests the construction problem is harder than expected")
        print(f"   or there are bugs in the fundamental approach")

Testing if F trivially belongs to its own Omega set...
TRIVIAL OMEGA SET MEMBERSHIP TEST
Testing: Does F ∈ Ω_{F,X}^ε for ANY reasonable ε?
This should ALWAYS be true by definition!

==================== TEST: n=15, k=3, X_cols=9, ε=0.1 ====================
Generated:
  F: (15, 15), rank 3
  X: (15, 4), rank 4
  Actual delta: 3.57e-16
  Null space dimension: 11

Testing if F ∈ Ω_{F,X}^0.1:
  1. Rank constraint: rank(F) = 3 == 3 = rank(F) ✓
  2. Sketching: ||FX - FX||_F = 0.00e+00 ✓
  3. Distance: ||F - F||_2 = 0.00e+00 ≤ 0.1 ✓
  4. Near-symmetry: F is δ-near-symmetric by construction ✓

  RESULT: F ∈ Ω_{F,X}^0.1: ✅ YES
  ✅ SUCCESS: F trivially satisfies all Omega constraints!

  Testing if construction algorithms can find F...
    Strict Nullspace Construction: Generated 1 matrices
      Closest matrix to F: distance = 0.0000e+00
      🎉 Found matrix very close to F!
    ✅ At least one method generates matrices

==================== TEST: n=15, k=3, X_cols=13, ε=0.1 ====================

In [98]:
# Parameters (edit as needed)
n = 15
k = 3
delta = 1e-3
epsilon = 5.0
spread_factor = 2.0  # Or a loop over multiple if you wish

# Generate test matrix F and X
F, X, U, V, actual_delta, Q = generate_near_symmetric_matrix(
    n=n, k=k, delta=delta, spread_factor=spread_factor, seed=42
)

# Construct Omega set (robust, strict algorithm)
num_samples = 10000  # or more for more results
verbose = True

omega_matrices = construct_omega_set_strict_relaxed(F, X, epsilon, num_samples=num_samples, verbose=verbose)

# Test: Is F in Omega? Should be True
A0 = omega_matrices[0]
print("F ∈ Omega? (distance from F):", np.linalg.norm(A0 - F), "Rank:", np.linalg.matrix_rank(A0), "Orthogonality:", np.linalg.norm(X.T @ (A0-F)))

# Optional: Check all generated matrices
for i, A in enumerate(omega_matrices):
    print(f"Matrix {i}: ||A-F||_2 = {np.linalg.norm(A-F):.3g}, Rank = {np.linalg.matrix_rank(A)}, ||Xᵗ(A-F)||_F = {np.linalg.norm(X.T @ (A-F), 'fro'):.2e}")

F, X_full, U, V, actual_delta, Q = generate_near_symmetric_matrix(
    n=15, k=3, delta=1e-3, spread_factor=3, seed=42
)
X = X_full[:, :k+1]    # or whatever your X size is
epsilon = 0.1        # or whatever value you want

omega, max_dist, argmax = empirical_supremum_over_omega(F, X, epsilon, num_samples=10000, verbose=True)

Generated 1/10000 valid Omega members.
F ∈ Omega? (distance from F): 0.0 Rank: 3 Orthogonality: 0.0
Matrix 0: ||A-F||_2 = 0, Rank = 3, ||Xᵗ(A-F)||_F = 0.00e+00
Generated 1/10000 valid Omega members.
Generated 1 Omega matrices.
Empirical supremum ||A-B||_2 = 0.0000e+00 over 10000 Omega samples.
Pair achieving supremum: (0, 0)
